# Python Fortgeschritten: C/C++ Extending und Embedding
## Tag 5 - Notebook 29
***
In diesem Notebook wird behandelt:
- ctypes Grundlagen
- ctypes Beispiele
- Cython Einführung
- Anwendungsfälle und Performance
***


## 1 ctypes Grundlagen

`ctypes` ermöglicht den Aufruf von C-Funktionen aus Python:
- **Bibliotheken laden**: `cdll.LoadLibrary()`, `windll` (Windows), `oledll` (Windows OLE)
- **Datentypen**: `c_int`, `c_float`, `c_char_p`, `c_void_p`, etc.
- **Funktionsaufrufe**: Direkter Aufruf von C-Funktionen
- **Zeiger**: `byref()`, `pointer()` für Zeiger-Argumente


In [1]:
from ctypes import cdll, c_int, c_float, c_char_p, c_void_p, byref
import sys

# Standard-Bibliothek verwenden (funktioniert auf allen Systemen)
# Beispiel: math.h Funktionen über libm (Unix) oder msvcrt (Windows)

if sys.platform.startswith('win'):
    # Windows: msvcrt.dll
    libc = cdll.msvcrt
else:
    # Unix/Linux: libc
    libc = cdll.LoadLibrary('libc.so.6')

# Beispiel: abs() Funktion
result = libc.abs(-42)
print(f"abs(-42) = {result}")

# Datentypen
print(f"\nctypes Datentypen:")
print(f"c_int: {c_int}")
print(f"c_float: {c_float}")
print(f"c_char_p: {c_char_p}")

# Beispiel mit c_int
value = c_int(42)
print(f"\nc_int(42): {value}")
print(f"Wert: {value.value}")


abs(-42) = 42

ctypes Datentypen:
c_int: <class 'ctypes.c_long'>
c_float: <class 'ctypes.c_float'>
c_char_p: <class 'ctypes.c_char_p'>

c_int(42): c_long(42)
Wert: 42


## 2 ctypes Beispiele

Praktische Beispiele mit ctypes:
- **Standard-Bibliotheken**: Zugriff auf System-Bibliotheken
- **Funktionssignaturen**: Definieren von Argument- und Rückgabetypen
- **Strukturen**: C-Strukturen mit `Structure` Klasse


In [2]:
from ctypes import Structure, c_int, c_char, POINTER

# C-Struktur definieren
class Point(Structure):
    _fields_ = [("x", c_int),
                ("y", c_int)]

# Struktur verwenden
p = Point(10, 20)
print(f"Point: x={p.x}, y={p.y}")

# Funktionssignaturen definieren
# Beispiel: hypot() aus math.h (falls verfügbar)
try:
    if sys.platform.startswith('win'):
        libm = cdll.LoadLibrary('msvcrt.dll')
    else:
        libm = cdll.LoadLibrary('libm.so.6')
    
    # Funktionssignatur definieren
    libm.hypot.argtypes = [c_float, c_float]
    libm.hypot.restype = c_float
    
    result = libm.hypot(c_float(3.0), c_float(4.0))
    print(f"\nhypot(3, 4) = {result}")
except (OSError, AttributeError):
    print("\nmath-Bibliothek nicht verfügbar, verwende Python-Alternative")
    import math
    print(f"math.hypot(3, 4) = {math.hypot(3, 4)}")


Point: x=10, y=20

math-Bibliothek nicht verfügbar, verwende Python-Alternative
math.hypot(3, 4) = 5.0


## 3 Cython Einführung

Cython ist eine Programmiersprache, die Python-ähnlichen Code zu C kompiliert:
- **Python-ähnliche Syntax**: Meist kompatibel mit Python
- **C-Performance**: Kompiliert zu C für Geschwindigkeit
- **Typannotationen**: Optionale Typen für bessere Performance
- **.pyx Dateien**: Cython-Quellcode wird zu C kompiliert

### Grundkonzepte

- **cdef**: C-Variablen und -Funktionen definieren
- **cpdef**: Funktionen, die von Python und C aufgerufen werden können
- **Typen**: `int`, `double`, `float` statt Python-Objekte


In [ ]:
# Cython-Beispiel (konzeptionell - würde normalerweise in .pyx Datei sein)
# 
# # Beispiel-Cython-Code (hypothetisch):
# # cython_example.pyx
# 
# cdef double fast_add(double a, double b):
#     return a + b
# 
# cpdef double add_numbers(double a, double b):
#     return fast_add(a, b)
# 
# def python_wrapper(a, b):
#     return add_numbers(a, b)
#
# Kompilierung:
# 1. cython cython_example.pyx -> cython_example.c
# 2. gcc -shared -pthread -fPIC -fwrapv -O2 -o cython_example.so cython_example.c
# 3. In Python: from cython_example import add_numbers

print("Cython-Beispiel (konzeptionell):")
print("Cython kompiliert Python-ähnlichen Code zu C")
print("Vorteile: Nahezu C-Performance bei Python-ähnlicher Syntax")
print("\nTypisches Cython-Pattern:")
print("  cdef int fast_function(int x):  # C-Funktion")
print("      return x * 2")
print("  ")
print("  def python_function(x):  # Python-Schnittstelle")
print("      return fast_function(x)")



## 4 Anwendungsfälle und Performance

### Wann ctypes verwenden?

- **Vorhandene C-Bibliotheken**: Wenn bereits C-Code existiert
- **System-Bibliotheken**: Zugriff auf OS-Funktionen
- **Einfache Integration**: Schneller Einstieg ohne Kompilierung
- **Nachteile**: Langsamer als nativer C-Code, Fehleranfällig

### Wann Cython verwenden?

- **Performance-kritischer Code**: Numerische Berechnungen, Schleifen
- **Python-Code optimieren**: Bestehenden Python-Code beschleunigen
- **Komplexe Algorithmen**: Wo Python zu langsam ist
- **Nachteile**: Kompilierung erforderlich, Build-Setup nötig

### Performance-Vergleich

Typische Geschwindigkeitsverbesserungen:
- **Python**: Referenz (1x)
- **ctypes**: 2-5x schneller (Overhead durch Python-C-Brücke)
- **Cython**: 10-100x schneller (abhängig vom Code)
- **Reines C**: 100-1000x schneller (ohne Python-Overhead)


In [ ]:
# Performance-Vergleich (konzeptionell)
import time

# Python-Version
def python_sum(n):
    total = 0
    for i in range(n):
        total += i
    return total

# Test
n = 1000000
start = time.time()
result_python = python_sum(n)
time_python = time.time() - start

print(f"Python: {time_python:.4f} Sekunden für Summe bis {n}")
print(f"Ergebnis: {result_python}")

print("\nHinweis: Mit Cython könnte diese Funktion 10-50x schneller sein")
print("durch Verwendung von cdef int statt Python-Integers")
print("\nBeispiel-Cython-Code:")
print("  cdef long cython_sum(long n):")
print("      cdef long total = 0")
print("      cdef long i")
print("      for i in range(n):")
print("          total += i")
print("      return total")


## 5 Zusammenfassung

### ctypes vs. Cython

| Aspekt | ctypes | Cython |
|--------|--------|--------|
| **Syntax** | Python | Python-ähnlich |
| **Kompilierung** | Nicht nötig | Erforderlich |
| **Performance** | 2-5x | 10-100x |
| **Verwendung** | C-Bibliotheken aufrufen | Python-Code optimieren |
| **Komplexität** | Niedrig | Mittel-Hoch |

### Wann verwenden?

- **ctypes**: Schnelle Integration vorhandener C-Bibliotheken
- **Cython**: Performance-Optimierung von Python-Code
- **Beide**: Wissenschaftliche Berechnungen, numerische Algorithmen


In [ ]:
# Zusammenfassung der wichtigsten Punkte
print("C/C++ Integration in Python:")
print("\n1. ctypes:")
print("   - Einfach zu verwenden")
print("   - Keine Kompilierung nötig")
print("   - Gut für System-Bibliotheken")
print("\n2. Cython:")
print("   - Höchste Performance")
print("   - Python-ähnliche Syntax")
print("   - Ideal für numerische Berechnungen")
print("\n3. Weitere Optionen:")
print("   - CFFI: Modernere Alternative zu ctypes")
print("   - pybind11: C++ Integration")
print("   - SWIG: Automatische Wrapper-Generierung")


## 7 Aufgaben

### Aufgabe (a): ctypes Grundlagen

Verwende ctypes für einfache C-Funktionen:

**Anforderungen:**
- Lade die Standard-C-Bibliothek (verwende `cdll.msvcrt` auf Windows oder `cdll.LoadLibrary('libc.so.6')` auf Linux)
- Rufe die `abs()` Funktion auf mit verschiedenen Werten (z.B. -42, 100, -5)
- Definiere eine C-Struktur `Point` mit zwei Integer-Feldern `x` und `y` (verwende `ctypes.Structure`)
- Erstelle zwei `Point`-Instanzen mit verschiedenen Werten
- Gib die Ergebnisse aus

**Tipp:** Verwende `from ctypes import cdll, Structure, c_int`. Für Strukturen: `class Point(Structure): _fields_ = [("x", c_int), ("y", c_int)]`

In [ ]:
# Deine Lösung

### Aufgabe (b): ctypes mit Funktionssignaturen

Definiere Funktionssignaturen für C-Funktionen:

**Anforderungen:**
- Lade die C-Bibliothek (libc/msvcrt)
- Versuche, eine mathematische Funktion wie `sqrt()` oder `pow()` aufzurufen (falls verfügbar)
- Definiere die Argument- und Rückgabetypen für die Funktion (verwende `.argtypes` und `.restype`)
- Rufe die Funktion mit korrekten Typen auf
- Gib die Ergebnisse aus

**Tipp:** Verwende `c_float` oder `c_double` für Fließkommazahlen. Falls die Funktion nicht verfügbar ist, verwende stattdessen Python's `math` Modul und erkläre den Unterschied.

In [ ]:
# Deine Lösung

### Aufgabe (c): Performance-Vergleich

Vergleiche Python vs. hypothetische C-Performance:

**Anforderungen:**
- Implementiere eine einfache Summen-Funktion in Python, die alle Zahlen von 0 bis n summiert
- Miss die Ausführungszeit für große Zahlen (z.B. n = 1.000.000 oder 10.000.000)
- Erkläre, wie Cython oder C diesen Code beschleunigen könnte
- Gib die Timing-Ergebnisse aus

**Tipp:** Verwende `time.time()` oder `time.perf_counter()` für Timing. Erkläre die Konzepte von `cdef` in Cython für Typannotationen.

In [ ]:
# Deine Lösung

### Aufgabe (d): Cython-Konzept (theoretisch)

Erkläre Cython-Konzepte:

**Anforderungen:**
- Beschreibe den Unterschied zwischen `cdef`, `cpdef` und `def` in Cython
- Zeige ein Beispiel-Pattern für Cython-Code (als Kommentar oder Pseudocode)
- Erkläre, wann Cython sinnvoll ist und wann nicht
- Gib die Erklärungen aus

**Tipp:** 
- `cdef`: C-Funktion, nur von Cython-Code aufrufbar, schnellste Performance
- `cpdef`: Kann von Python und Cython aufgerufen werden, gute Performance
- `def`: Python-Funktion, langsamste, aber vollständig Python-kompatibel

In [ ]:
# Deine Lösung

### Lösungen

In [ ]:
from ctypes import cdll, Structure, c_int, c_float, c_double
import sys
import time
import math
import logging

logging.basicConfig(level=logging.DEBUG, format='%(levelname)s: %(message)s')

# Musterlösung (a)
logging.debug("=== Aufgabe (a): ctypes Grundlagen ===")

# C-Bibliothek laden
if sys.platform.startswith('win'):
    libc = cdll.msvcrt
    logging.debug("Windows: msvcrt.dll geladen")
else:
    try:
        libc = cdll.LoadLibrary('libc.so.6')
        logging.debug("Linux: libc.so.6 geladen")
    except OSError:
        libc = cdll.LoadLibrary('libc.dylib')
        logging.debug("macOS: libc.dylib geladen")

# abs() aufrufen
result1 = libc.abs(-42)
result2 = libc.abs(100)
result3 = libc.abs(-5)
logging.debug(f"abs(-42) = {result1}")
logging.debug(f"abs(100) = {result2}")
logging.debug(f"abs(-5) = {result3}")

# C-Struktur definieren
class Point(Structure):
    _fields_ = [("x", c_int),
                ("y", c_int)]

# Point-Instanzen erstellen
p1 = Point(10, 20)
p2 = Point(5, 15)
logging.debug(f"\nPoint 1: x={p1.x}, y={p1.y}")
logging.debug(f"Point 2: x={p2.x}, y={p2.y}")

# Musterlösung (b)
logging.debug("\n=== Aufgabe (b): ctypes mit Funktionssignaturen ===")

try:
    # Versuche sqrt() zu verwenden
    if sys.platform.startswith('win'):
        libm = cdll.LoadLibrary('msvcrt.dll')
    else:
        libm = cdll.LoadLibrary('libm.so.6')
    
    # Funktionssignatur definieren
    libm.sqrt.argtypes = [c_double]
    libm.sqrt.restype = c_double
    
    result_sqrt = libm.sqrt(c_double(25.0))
    logging.debug(f"sqrt(25.0) mit ctypes = {result_sqrt}")
except (OSError, AttributeError):
    # Fallback auf Python math
    logging.debug("C-Bibliothek nicht verfügbar, verwende Python math")
    result_sqrt = math.sqrt(25.0)
    logging.debug(f"sqrt(25.0) mit Python math = {result_sqrt}")
    logging.debug("Hinweis: ctypes ermöglicht direkten Aufruf von C-Funktionen mit definierten Typen")

# Musterlösung (c)
logging.debug("\n=== Aufgabe (c): Performance-Vergleich ===")

def python_sum(n):
    """Python-Version der Summen-Funktion"""
    total = 0
    for i in range(n):
        total += i
    return total

# Timing für verschiedene n-Werte
for n in [1000000, 10000000]:
    start = time.perf_counter()
    result = python_sum(n)
    elapsed = time.perf_counter() - start
    logging.debug(f"Python sum({n}): {elapsed:.4f} Sekunden, Ergebnis: {result}")

logging.debug("\nCython könnte diese Funktion beschleunigen durch:")
logging.debug("  - cdef int statt Python int: cdef long total = 0")
logging.debug("  - cdef long i: for i in range(n):")
logging.debug("  - Kompilierung zu C-Code")
logging.debug("  - Erwartete Beschleunigung: 10-50x schneller")

# Beispiel-Cython-Code als Kommentar
logging.debug("\nBeispiel-Cython-Code (konzeptionell):")
logging.debug("  cdef long cython_sum(long n):")
logging.debug("      cdef long total = 0")
logging.debug("      cdef long i")
logging.debug("      for i in range(n):")
logging.debug("          total += i")
logging.debug("      return total")

# Musterlösung (d)
logging.debug("\n=== Aufgabe (d): Cython-Konzept (theoretisch) ===")

logging.debug("\nUnterschied zwischen cdef, cpdef und def:")
logging.debug("\n1. cdef:")
logging.debug("   - C-Funktion, nur von Cython-Code aufrufbar")
logging.debug("   - Schnellste Performance (direkt kompiliert zu C)")
logging.debug("   - Nicht von Python aus aufrufbar")
logging.debug("   - Beispiel: cdef int fast_function(int x):")

logging.debug("\n2. cpdef:")
logging.debug("   - Kann von Python UND Cython aufgerufen werden")
logging.debug("   - Gute Performance (C-Version für Cython, Python-Wrapper für Python)")
logging.debug("   - Beispiel: cpdef int hybrid_function(int x):")

logging.debug("\n3. def:")
logging.debug("   - Python-Funktion, vollständig Python-kompatibel")
logging.debug("   - Langsamste Performance (Python-Overhead)")
logging.debug("   - Beispiel: def python_function(x):")

logging.debug("\nWann Cython verwenden:")
logging.debug("  - Performance-kritischer Code (numerische Berechnungen, Schleifen)")
logging.debug("  - Bestehender Python-Code optimieren")
logging.debug("  - Komplexe Algorithmen, wo Python zu langsam ist")
logging.debug("  - Integration mit C-Bibliotheken")

logging.debug("\nWann Cython NICHT verwenden:")
logging.debug("  - Einfache Skripte ohne Performance-Anforderungen")
logging.debug("  - Code, der hauptsächlich I/O-Operationen macht")
logging.debug("  - Wenn Build-Setup zu komplex wird")
